In [4]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import *

from sklearn.model_selection import train_test_split,StratifiedShuffleSplit,KFold,GroupKFold,StratifiedGroupKFold,GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,StackingClassifier,BaggingClassifier
from xgboost import XGBClassifier

In [5]:
dataTrain=pd.read_csv("data/Train.txt",sep="#|;",engine="python",header=None,index_col = False)
dataTest=pd.read_csv("data/Test.txt",sep="#|;",engine="python",header=None,index_col = False)
dataCros=pd.read_csv("data/Cross_Validation.txt",sep="#|;",engine="python",header=None,index_col = False)

data=pd.concat([dataTrain,dataTest,dataCros])

In [3]:



data.describe()

,0,1,2,3,4,5,6,7,8,9,...,537,538,539,540,541,542,543,544,545,546
count,20189.000000,20189.000000,20189.000000,20189.000000,20189.000000,20189.000000,20189.000000,20189.000000,20189.000000,20189.000000,...,20189.000000,20189.000000,20189.000000,20189.000000,20189.000000,20189.000000,20189.000000,20189.000000,20189.000000,20189.000000
mean,11.224479,-3.423828,-1.005713,1.706400,-1.159582,-0.855778,-1.103048,-0.267441,-0.128349,0.159302,...,-0.303256,0.773173,-0.236877,0.099867,0.113846,0.347402,0.046918,0.146214,0.026270,4.104033
std,6.941206,12.906586,6.641375,5.304308,4.364610,3.429600,2.600222,2.251118,1.959483,1.809170,...,3.432375,2.807844,2.592954,2.329421,1.958721,1.758277,1.606191,1.455374,1.358208,1.047831
min,0.000000,-41.387033,-25.716292,-16.649619,-16.727334,-14.097185,-11.150499,-10.332489,-9.643177,-9.146440,...,-15.739322,-12.423641,-10.907989,-14.311366,-9.523313,-8.734160,-7.556286,-5.533654,-8.683989,-0.528020
25%,5.000000,-13.156473,-5.589819,-2.133137,-3.958086,-3.075181,-2.867972,-1.727710,-1.367883,-0.974871,...,-2.618182,-1.118800,-1.925435,-1.350946,-1.086751,-0.725954,-0.952058,-0.771705,-0.825883,3.514669
50%,11.000000,-1.369778,-1.044129,1.768324,-0.981802,-0.777229,-1.086701,-0.218771,-0.067052,0.209278,...,-0.208494,0.889173,-0.080375,0.239705,0.201000,0.377365,0.078007,0.169320,0.056811,4.168822
75%,17.000000,6.766844,3.764928,5.421992,1.693227,1.461983,0.679469,1.242841,1.138865,1.347726,...,2.165309,2.715580,1.540627,1.672862,1.382336,1.474741,1.087508,1.099771,0.911312,4.761209
max,23.000000,28.907357,19.357245,21.135432,20.982353,11.971851,14.983369,10.504401,9.182450,8.129699,...,15.025263,10.739319,11.194952,9.769175,11.536425,7.482802,7.454840,6.677344,5.998552,7.183624


In [6]:

scaler=StandardScaler()
X=data.drop(0,axis=1)
X=scaler.fit_transform(X)
Y=data[0]

In [ ]:
%%time
def find_best_model(learning_algorithm,X,Y,expected_accuracy=0.75,model_selection=None,model_selection_params={},params={}): 
    while True:
        for (train_size,rand) in zip(np.random.random(50),np.arange(1,50)):
            train_size=round((train_size+0.1)*0.8,2)
            X_train,X_test,y_train,y_test = train_test_split(X,Y,train_size=train_size,random_state=rand)
            
            
            #adding default parametrs If params is not empty and model_selction is not None
            if model_selection:
                for (key,value) in params.items():
                    model = learning_algorithm()
                    l=set()
                    if hasattr(value,"__iter__") and type(value)!=str:
                        l=set(value)
                    else:
                        l.add(value)
                    l.add(model.__dict__.get(key))
                    params[key]=list(l)
                print(params)
                model_selection = model_selection(model,params,scoring="accuracy",**model_selection_params)
                model = model_selection.fit(X_train,y_train)
            else:
                model = learning_algorithm(**params)
                model.fit(X_train,y_train)

            X_val,X_test,y_val,y_test=train_test_split(X_test,y_test,train_size=0.50,random_state=1)

            y_val_predict=model.predict(X_val)
            y_test_predict=model.predict(X_test)

            val_accuracy = accuracy_score(y_val_predict,y_val)
            test_accuracy=accuracy_score(y_test_predict,y_test)

            print((train_size,rand),f"Model: {learning_algorithm.__name__}, Validation Accuracy:{val_accuracy:.3}, Test Accuracy:{test_accuracy:.3}")
            if val_accuracy>=expected_accuracy and test_accuracy>=expected_accuracy:
                return test_accuracy,model

accuracy,model=find_best_model(DecisionTreeClassifier,X,Y,params={'criterion': 'entropy'},model_selection=RandomizedSearchCV)

{'criterion': ['gini', 'entropy']}


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 2 is smaller than n_iter=10. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [15]:
type("a")==str

True

In [9]:
param_grid =   {'penalty' : ['l1', 'l2', 'elasticnet', None],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter' : [100, 1000,2500, 5000]
    }
accuracy,model=find_best_model(LogisticRegression,X,Y,params={'max_iter': 300},expected_accuracy=0.92)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.34, 1) Model: LogisticRegression, Validation Accuracy:0.918, Test Accuracy:0.911


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.86, 2) Model: LogisticRegression, Validation Accuracy:0.91, Test Accuracy:0.919


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.24, 3) Model: LogisticRegression, Validation Accuracy:0.916, Test Accuracy:0.907


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.2, 4) Model: LogisticRegression, Validation Accuracy:0.905, Test Accuracy:0.908
(0.69, 5) Model: LogisticRegression, Validation Accuracy:0.924, Test Accuracy:0.924


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [10]:
import pickle
with open("logistic_regression.pkl","wb") as file:
    pickle.dump(model,file)

In [ ]:
params={'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['log2', 'sqrt',None],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],
 "criterion":["gini", "entropy", "log_loss"]
}

accuracy,model=find_best_model(RandomForestClassifier,X,Y,expected_accuracy=0.97)

(0.32, 1) Model: RandomForestClassifier, Validation Accuracy:0.954, Test Accuracy:0.95
(0.08, 2) Model: RandomForestClassifier, Validation Accuracy:0.896, Test Accuracy:0.888
(0.46, 3) Model: RandomForestClassifier, Validation Accuracy:0.962, Test Accuracy:0.961
(0.68, 4) Model: RandomForestClassifier, Validation Accuracy:0.973, Test Accuracy:0.966
(0.45, 5) Model: RandomForestClassifier, Validation Accuracy:0.961, Test Accuracy:0.959
(0.32, 6) Model: RandomForestClassifier, Validation Accuracy:0.954, Test Accuracy:0.95
(0.51, 7) Model: RandomForestClassifier, Validation Accuracy:0.965, Test Accuracy:0.963
(0.79, 8) Model: RandomForestClassifier, Validation Accuracy:0.969, Test Accuracy:0.975
(0.53, 9) Model: RandomForestClassifier, Validation Accuracy:0.962, Test Accuracy:0.966
(0.22, 10) Model: RandomForestClassifier, Validation Accuracy:0.937, Test Accuracy:0.939
(0.18, 11) Model: RandomForestClassifier, Validation Accuracy:0.931, Test Accuracy:0.931
(0.44, 12) Model: RandomForestCl

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,train_size=0.66)          
model = LogisticRegression()
model.fit(X_train,y_train)
X_val,X_test,y_val,y_test=train_test_split(X_test,y_test,train_size=0.70,random_state=1)
y_val_predict=model.predict(X_val)
y_test_predict=model.predict(X_test)
val_accuracy = accuracy_score(y_val_predict,y_val)
test_accuracy=accuracy_score(y_test_predict,y_test)
print(f"Validation Accuracy:{val_accuracy:.3}, Test Accuracy:{test_accuracy:.3}")

In [1]:
import pickle
with open("randomforest.pkl","wb") as file:
    pickle.dump(model,file)

NameError: name 'model' is not defined

In [ ]:
find_best_model(XGBClassifier,X,Y,expected_accuracy=0.96)

(0.22, 1) Model: XGBClassifier, Validation Accuracy:0.914, Test Accuracy:0.922
(0.35, 2) Model: XGBClassifier, Validation Accuracy:0.937, Test Accuracy:0.936


In [ ]:
pkl_file = open('randomforest.pkl', 'rb')
m=pickle.load(pkl_file)

In [ ]:
Y_predict=m.predict(X)
accuracy_score(Y_predict,Y)

In [ ]:
DecisionTreeClassifier.__name__

In [ ]:
estimator,
    param_grid,
    *,
    scoring=None,
    n_jobs=None,
    refit=True,
    cv=None,
    verbose=0,
    pre_dispatch='2*n_jobs',
    error_score=nan,
    return_train_score=False,
)

In [ ]:
{**DecisionTreeClassifier().__dict__,'criterion': 'entropy'}

In [ ]:
RandomizedSearchCV()

In [ ]:
model = DecisionTreeClassifier()
params = {"random_state": 2}
for (key,value) in params.items():
    if hasattr(value,"__iter__"):
        l=list(value)
    else:
        l=[value]
    l.append(model.__dict__.get(key))
    params[key]=l
                
                

In [ ]:
DecisionTreeClassifier(**{})

In [ ]:
dir(())

In [ ]:
hasattr([],"__iter__")

In [ ]:
print(model.__dict__.get("estimator5"))

In [ ]:
DecisionTreeClassifier().__dict__

In [15]:
set(Y.value_counts().index)

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23}

In [18]:
import threading

threading.Thread.run(find_best_model(BaggingClassifier,X,Y,expected_accuracy=0.8))

(0.35, 1) Model: BaggingClassifier, Validation Accuracy:0.874, Test Accuracy:0.872


AttributeError: 'tuple' object has no attribute '_target'

In [ ]:
print("sd")

In [20]:
dir(set())

['__and__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__iand__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__isub__',
 '__iter__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__or__',
 '__rand__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__ror__',
 '__rsub__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__xor__',
 'add',
 'clear',
 'copy',
 'difference',
 'difference_update',
 'discard',
 'intersection',
 'intersection_update',
 'isdisjoint',
 'issubset',
 'issuperset',
 'pop',
 'remove',
 'symmetric_difference',
 'symmetric_difference_update',
 'union',
 'update']

In [25]:
a=set([1,2,3,4])
b=set([1,2,3,5,6,7])
a.union(b)

{1, 2, 3, 4, 5, 6, 7}

In [24]:
a

{4, 5, 6, 7}